In [2]:

vcf_file_address = "22_5sample.vcf"  # "22_4k.vcf" 
vcf_file = open(vcf_file_address,'r');

blockid_blocks_s1 = []
allele_blocks_s1 = []
varpos_blocks_s1 = []


blockid_blocks_s2 = []
allele_blocks_s2 = []
varpos_blocks_s2 = []

blockid_blocks_s3 = []
allele_blocks_s3 = []
varpos_blocks_s3 = []

blockid_blocks_s4 = []
allele_blocks_s4 = []
varpos_blocks_s4 = []

var_blockids1_blocks_s2 = []   # for each var in s2, shows the s1 blockid
var_blockids1_blocks_s3 = []   # for each var in s3, shows the s1 blockid
var_blockids1_blocks_s4 = []   # for each var in s4, shows the s1 blockid


# for parental that it's only one block per chr so it doesn't have blockid
alleles_s5 = []
varposs_s5 = []


varposs_all = []

num_weird_occurs=0

var_lines_list = []

header_lines_list = []
for line in vcf_file:
    line_strip = line.strip()
    
    if line_strip.startswith('#'):
        header_lines_list.append(line_strip)
        sample_names = line_strip.split('\t')[9:]   # last line of header contains sample name
        #print(sample_names)
        
    else:
        line_parts = line_strip.split('\t')
        var_lines_list.append(line_parts)
        ref_allele=line_parts[3]
        alt_allele=line_parts[4]
        
        
        
        #gt_flags, s1, sss, ss, s2, s5 = line_parts[8:]
        gt_flags, s1, s2,s3,s4, s5 = line_parts[8:]       # hifi-whatshp illumina-whatshp hifi-hapcut illumina-hapcut true

        varpos = int(line_parts[1])                 # variant position
        
        varposs_all.append(varpos)
        gt_flags_split = gt_flags.split(":")
        
        if "PS" in gt_flags_split and len(ref_allele)==1 and len(alt_allele)==1:  # if there is inconsitency in genotypes, it will be multi allelelic 
            idx_ps = gt_flags.split(":").index("PS")
            idx_gt = gt_flags.split(":").index("GT")
            

                                                 ## s5- parental - last sample in the vcf file
            split_s5 = s5.split(":")
            allele_s5 = split_s5[idx_gt]
            blockid_s5 = split_s5[idx_ps]
            
            if allele_s5 == "0|1" or allele_s5 == "1|0":
                alleles_s5.append(int(allele_s5[0]))
                varposs_s5.append(varpos)

            
            
                                                ###  sample 1 ###
            split_s1 = s1.split(":")
            blockid_s1 = split_s1[idx_ps]
            allele_s1 = split_s1[idx_gt]
            

            if blockid_s1 == ".":                                 # unphased variants for sample one
                pass
            
            elif blockid_s1 in blockid_blocks_s1:
                assert allele_s1[1] == "|", "allele_s1[1] should be | and so phased. A non-phased variant is in the phase block"
                
                #if blockid_s1 == previous_blockid_s1: previous_blockid_s1_idx has been defined in previous variant which is the last idx
                if blockid_s1 != previous_blockid_s1:  
                    # weird situation happens for whatshap in which two variants are in the same block but in between there is/are other phaseblock(s)
                    # note that the unphased variants are ignored sooner and are not considered here
                    previous_blockid_s1_idx = blockid_blocks_s1.index(blockid_s1)
                    #print("two variants are in the same block, but far from each other",varpos)                
                    num_weird_occurs+=1

                allele_blocks_s1[previous_blockid_s1_idx].append( int(allele_s1[0]) )
                varpos_blocks_s1[previous_blockid_s1_idx].append( varpos ) 

                
            else:                                                  # A new block started, it can be the first block
                
                allele_blocks_s1.append( [int(allele_s1[0])] )     # alleles of a phase block (all vars are hetrozygous) 
                varpos_blocks_s1.append( [varpos] )                # variant posistion of a block
                blockid_blocks_s1.append( blockid_s1 )             # add the blockid of the first var  to the list of blockids
                
                previous_blockid_s1 = blockid_s1
                previous_blockid_s1_idx = len(blockid_blocks_s1)-1  # the index of the last block id
                


                
       
    
                                                ###  sample 2 ###
            split_s2 = s2.split(":")
            blockid_s2 = split_s2[idx_ps]
            allele_s2 = split_s2[idx_gt]
            if blockid_s2 == ".":
                pass
            elif blockid_s2 in blockid_blocks_s2:
                assert allele_s2[1] == "|", "allele_s2[1] should be | and so phased. A non-phased variant is in the phase block"
                
                if blockid_s2 != previous_blockid_s2:  
                    previous_blockid_s2_idx = blockid_blocks_s2.index(blockid_s2)
                    num_weird_occurs+=1
                allele_blocks_s2[previous_blockid_s2_idx].append( int(allele_s2[0]) )
                varpos_blocks_s2[previous_blockid_s2_idx].append( varpos ) 
                var_blockids1_blocks_s2[previous_blockid_s2_idx].append( blockid_s1 )
                
            else:                                                 # A new block started   (it can be the first block)
                allele_blocks_s2.append( [int(allele_s2[0])] )     # alleles of a phase block (all vars are hetrozygous) 
                varpos_blocks_s2.append( [varpos] )                # variant posistion of a block
                blockid_blocks_s2.append( blockid_s2 )             # add the blockid of the first var  to the list of blockids
                var_blockids1_blocks_s2.append( [blockid_s1] )  
                previous_blockid_s2 = blockid_s2
                previous_blockid_s2_idx = len(blockid_blocks_s2)-1   # the index of the last block id
                

                                                ###  sample 3 ###
            split_s3 = s3.split(":")
            blockid_s3 = split_s3[idx_ps]
            allele_s3 = split_s3[idx_gt]
            if blockid_s3 == ".":
                pass
            elif blockid_s3 in blockid_blocks_s3:
                assert allele_s3[1] == "|", "allele_s3[1] should be | and so phased. A non-phased variant is in the phase block"
                
                if blockid_s3 != previous_blockid_s3:  
                    previous_blockid_s3_idx = blockid_blocks_s3.index(blockid_s3)
                    num_weird_occurs+=1
                allele_blocks_s3[previous_blockid_s3_idx].append( int(allele_s3[0]) )
                varpos_blocks_s3[previous_blockid_s3_idx].append( varpos ) 
                var_blockids1_blocks_s3[previous_blockid_s3_idx].append( blockid_s1 )
                
            else:                                                 # A new block started   (it can be the first block)
                allele_blocks_s3.append( [int(allele_s3[0])] )     # alleles of a phase block (all vars are hetrozygous) 
                varpos_blocks_s3.append( [varpos] )                # variant posistion of a block
                blockid_blocks_s3.append( blockid_s3 )             # add the blockid of the first var  to the list of blockids
                var_blockids1_blocks_s3.append( [blockid_s1] )  
                previous_blockid_s3 = blockid_s3
                previous_blockid_s3_idx = len(blockid_blocks_s3)-1   # the index of the last block id
                
   
    
                                                ###  sample 3 ###
            split_s4 = s4.split(":")
            blockid_s4 = split_s4[idx_ps]
            allele_s4 = split_s4[idx_gt]
            if blockid_s4 == ".":
                pass
            elif blockid_s4 in blockid_blocks_s4:
                assert allele_s4[1] == "|", "allele_s4[1] should be | and so phased. A non-phased variant is in the phase block"
                
                if blockid_s4 != previous_blockid_s4:  
                    previous_blockid_s4_idx = blockid_blocks_s4.index(blockid_s4)
                    num_weird_occurs+=1
                allele_blocks_s4[previous_blockid_s4_idx].append( int(allele_s4[0]) )
                varpos_blocks_s4[previous_blockid_s4_idx].append( varpos ) 
                var_blockids1_blocks_s4[previous_blockid_s4_idx].append( blockid_s1 )
                
            else:                                                 # A new block started   (it can be the first block)
                allele_blocks_s4.append( [int(allele_s4[0])] )     # alleles of a phase block (all vars are hetrozygous) 
                varpos_blocks_s4.append( [varpos] )                # variant posistion of a block
                blockid_blocks_s4.append( blockid_s4 )             # add the blockid of the first var  to the list of blockids
                var_blockids1_blocks_s4.append( [blockid_s1] )  
                previous_blockid_s4 = blockid_s4
                previous_blockid_s4_idx = len(blockid_blocks_s4)-1   # the index of the last block id
                




print("Num phaseblocks in sample one is ",len(allele_blocks_s1))
print("Num phaseblocks in sample two is ",len(allele_blocks_s2))
print("Num phaseblocks in sample three is ",len(allele_blocks_s3))
print("Num phaseblocks in sample four is ",len(allele_blocks_s4))
print("Num vars in sample five-parental is ",len(varposs_s5))
print("Num all vars is ",len(varposs_all))

print("number of occurance of weird case (=two variants are in the same block, but far from each other) ", num_weird_occurs)
 

Num phaseblocks in sample one is  252
Num phaseblocks in sample two is  251
Num phaseblocks in sample three is  6771
Num phaseblocks in sample four is  6895
Num vars in sample five-parental is  20903
Num all vars is  71923
number of occurance of weird case (=two variants are in the same block, but far from each other)  535


In [3]:
#blockid_blocks_s3[-1], varpos_blocks_s3[-1], allele_blocks_s3[-1]

In [4]:

# re-ordering  the phase block allele based on parent

## important ## 
# if we don't have the parental , how we can reorder

flipped_s1=[]
flipped_s2=[]
flipped_s3=[]
flipped_s4=[]


reordered_block_i_list_s1=[]
reordered_block_i_list_s2=[]
reordered_block_i_list_s3=[]
reordered_block_i_list_s4=[]

allele_blocks_s1_reordered = []
allele_blocks_s2_reordered = []
allele_blocks_s3_reordered = []
allele_blocks_s4_reordered = []



number_phaseblocks_s1= len(allele_blocks_s1)

for block_i in range(number_phaseblocks_s1): # 
    allele_block_s1=allele_blocks_s1[block_i]
    varpos_block_s1=varpos_blocks_s1[block_i]

    similiar_count=0
    diss_count=0
    for var_i, varpos in enumerate(varpos_block_s1):
        allele_s1= allele_block_s1[var_i]
        if varpos in varposs_s5:
            allele_s5 = alleles_s5[varposs_s5.index(varpos)]
            if allele_s1 == allele_s5:
                similiar_count+=1
            else: 
                diss_count+=1
                
    if similiar_count < diss_count:
        allele_block_s1_reordered = [1-i for i in allele_block_s1] # converting 0->1 and 1->0
        allele_blocks_s1_reordered.append(allele_block_s1_reordered)
        reordered_block_i_list_s1.append(blockid_blocks_s1[block_i])
    else:
        allele_blocks_s1_reordered.append(allele_block_s1)
print("done1")    




number_phaseblocks_s2= len(allele_blocks_s2)

for block_i in range(number_phaseblocks_s2): # 
    allele_block_s2=allele_blocks_s2[block_i]
    varpos_block_s2=varpos_blocks_s2[block_i]
    
    similiar_count=0
    diss_count=0
    for var_i, varpos in enumerate(varpos_block_s2):
        allele_s2= allele_block_s2[var_i]
        
        if varpos in varposs_s5:
            allele_s5= alleles_s5[varposs_s5.index(varpos)]
            if allele_s2 == allele_s5:
                similiar_count+=1
            else: 
                diss_count+=1
                
    if similiar_count < diss_count:
        allele_block_s2_reordered = [1-i for i in allele_block_s2]
        allele_blocks_s2_reordered.append(allele_block_s2_reordered)
        reordered_block_i_list_s2.append(blockid_blocks_s2[block_i])
    else:
        allele_blocks_s2_reordered.append(allele_block_s2)
print("done2")     



number_phaseblocks_s3= len(allele_blocks_s3)
for block_i in range(number_phaseblocks_s3): # 
    allele_block_s3=allele_blocks_s3[block_i]
    varpos_block_s3=varpos_blocks_s3[block_i]
    
    similiar_count=0
    diss_count=0
    for var_i, varpos in enumerate(varpos_block_s3):
        allele_s3= allele_block_s3[var_i]
        
        if varpos in varposs_s5:
            allele_s5= alleles_s5[varposs_s5.index(varpos)]
            if allele_s3 == allele_s5:
                similiar_count+=1
            else: 
                diss_count+=1
    if similiar_count < diss_count:
        allele_block_s3_reordered = [1-i for i in allele_block_s3] # converting 0->1 and 1->0
        allele_blocks_s3_reordered.append(allele_block_s3_reordered)        
        reordered_block_i_list_s3.append(blockid_blocks_s3[block_i])
    else:
        allele_blocks_s3_reordered.append(allele_block_s3)
print("done3")     


number_phaseblocks_s4= len(allele_blocks_s4)
for block_i in range(number_phaseblocks_s4): # 
   
    allele_block_s4=allele_blocks_s4[block_i]
    varpos_block_s4=varpos_blocks_s4[block_i]

    similiar_count=0
    diss_count=0
    for var_i, varpos in enumerate(varpos_block_s4):
        allele_s4 = allele_block_s4[var_i]
        
        if varpos in varposs_s5:
            allele_s5= alleles_s5[varposs_s5.index(varpos)]
            if allele_s4 == allele_s5:
                similiar_count+=1
            else: 
                diss_count+=1
                
    if similiar_count < diss_count:
        allele_block_s4_reordered = [1-i for i in allele_block_s4] 
        allele_blocks_s4_reordered.append(allele_block_s4_reordered)
        
        reordered_block_i_list_s4.append(blockid_blocks_s4[block_i])
    else:
        allele_blocks_s4_reordered.append(allele_block_s4)
        
print("done4")    



done1
done2
done3
done4


In [5]:
#print(reordered_block_i_list_s3[20])


16205497


In [5]:
# #16069374
# block_id_s1='16205497'
# block_id_s2='16147398'
# block_id_s3='16205497'
# block_id_s4='16205497'

# print(block_id_s1 in reordered_block_i_list_s1)
# print(block_id_s2 in reordered_block_i_list_s2)
# print(block_id_s3 in reordered_block_i_list_s3)
# print(block_id_s4 in reordered_block_i_list_s4)

In [7]:


matrix_row_lists=[]      # cant we creat this list when parsing VCF ? No. becuase it's reordered.  
for varpos in varposs_all:
    # for increasing the speed we can use the information that the phase blocks are in order.
    # but not for those weird phase block


    allele_s1='.'
    blockid_s1='.'
    for block_i_s1 in range(len(blockid_blocks_s1)):
        allele_block_s1 = allele_blocks_s1_reordered[block_i_s1]
        varpos_block_s1 = varpos_blocks_s1[block_i_s1]
        if varpos in varpos_block_s1:
            allele_s1 = allele_block_s1[varpos_block_s1.index(varpos)]
            blockid_s1 = blockid_blocks_s1[block_i_s1]
            break # when var is found, no need to continue for loop
            
            
    allele_s2='.'
    blockid_s2='.'
    for block_i_s2 in range(len(blockid_blocks_s2)):
        allele_block_s2 = allele_blocks_s2_reordered[block_i_s2]
        varpos_block_s2 = varpos_blocks_s2[block_i_s2]
        if varpos in varpos_block_s2:
            allele_s2 = allele_block_s2[varpos_block_s2.index(varpos)]
            blockid_s2 = blockid_blocks_s2[block_i_s2]
            break

            
    allele_s3='.'
    blockid_s3='.'
    for block_i_s3 in range(len(blockid_blocks_s3)):
        allele_block_s3 = allele_blocks_s3_reordered[block_i_s3]
        varpos_block_s3 = varpos_blocks_s3[block_i_s3]

        if varpos in varpos_block_s3:
            allele_s3 = allele_block_s3[varpos_block_s3.index(varpos)]
            blockid_s3 = blockid_blocks_s3[block_i_s3]
            break 

            
    allele_s4='.'
    blockid_s4='.'
    for block_i_s4 in range(len(blockid_blocks_s4)):
        allele_block_s4 = allele_blocks_s4_reordered[block_i_s4]
        varpos_block_s4 = varpos_blocks_s4[block_i_s4]

        if varpos in varpos_block_s4:
            allele_s4 = allele_block_s4[varpos_block_s4.index(varpos)]
            blockid_s4 = blockid_blocks_s4[block_i_s4]
            break 

    
            
    if varpos in varposs_s5:
        allele_s5 = alleles_s5[varposs_s5.index(varpos)]
    else: allele_s5= '.'

    if not (allele_s1=='.' and allele_s2=='.' and allele_s3=='.' and allele_s4=='.' and allele_s5=='.'): 
        matrix_row_list=[str(varpos),str(allele_s1)+":"+str(blockid_s1),
                         str(allele_s2)+":"+str(blockid_s2),str(allele_s3)+":"+str(blockid_s3),
                         str(allele_s4)+":"+str(blockid_s4), str(allele_s5)] 
        matrix_row_lists.append(matrix_row_list)

        
# one option can be for loop over blockid_blocks_s1 instead of varposs_all, then search in  blockid_blocks_s1 also for printing sort them to overcome weird phase block
# the issue will be for those variant that are not phased in blockid_blocks_s1 but in blockid_blocks_s2

print("done")


done


In [8]:
matrix_row_lists[940]

['16242990', '.:.', '0:16147398', '0:16242485', '0:16242485', '.']

In [9]:

matrix_row_lists_prd = [] 
for row_i, matrix_row_list in enumerate(matrix_row_lists):
    
    varpos, s1, s2, s3, s4, allele_s5 = matrix_row_list
    # ['16240810', '1:16147398', '1:16147398', '0:16238215', '1:16238215', '.']
    
    all_alleles_raw=[s1[0], s2[0],s3[0],s4[0]]
    all_alleles=[i for i in all_alleles_raw if i != '.']   
    #  # allele_majvt=max(set(all_alleles), key=all_alleles.count)
    
    allele_majvt_withID=".:."
    allele_majvt="."
    if all_alleles: 
        if all_alleles.count('0') > all_alleles.count('1'):
            allele_majvt = '0'
        elif all_alleles.count('1') > all_alleles.count('0'):
            allele_majvt = '1'
        elif all_alleles.count('1') == all_alleles.count('0') and all_alleles.count('1')>1:
            allele_majvt = all_alleles[0] # s1
            
        if allele_majvt !=".":
            allele_majvt_withID=allele_majvt+":16050159"

    matrix_row_list_prd = [varpos, s1, s2, s3, s4, allele_s5, allele_majvt_withID] 
    
    matrix_row_lists_prd.append(matrix_row_list_prd)
   


In [10]:

matrix_address=vcf_file_address[:-4]+"_parent_v2d.txt"

matrix_file=open(matrix_address,'w');
matrix_file.write("\t\t".join(["Pos","HiFiWsp","HiFiHpct","IllmWsp","IllmHpct","parental","predicted"])+"\n")

for matrix_row_list in matrix_row_lists_prd:

        matrix_file.write("\t".join(matrix_row_list)+"\n")

matrix_file.close()
   
    
print("done")

done


In [11]:
matrix_row_lists_prd[10]
matrix_row_list_prd = matrix_row_lists_prd[3] 
allele_prd_blockId_prd = matrix_row_list_prd[6]
allele_prd, blockId_prd =allele_prd_blockId_prd.split(":")
allele_prd_blockId_prd, blockId_prd

('1:16050159', '16050159')

In [12]:
diff_list=[]

var_lines_list_prd=[]
num_het=0
varpos_list_inmatrix=[i[0] for i in matrix_row_lists_prd]
num1=0
for var_line_list in var_lines_list: # 
    var_line_list_part=var_line_list[:9]

    varpos = var_line_list_part[1]
    if varpos in varpos_list_inmatrix:
        
        ref_allele=var_line_list[4]
        alt_allele=var_line_list[4]
        
        
        gt_flags = var_line_list[8]
        idx_ps = gt_flags.split(":").index("PS")
        idx_gt = gt_flags.split(":").index("GT")   
        
        var_idx = varpos_list_inmatrix.index(varpos)        
        matrix_row_list_prd = matrix_row_lists_prd[var_idx]  
        #blockId_s1 = matrix_row_list_prd[1].split(":")[1]
        
        allele_prd_blockId_prd = matrix_row_list_prd[6] # ".:." or "0:123123"
            
        allele_prd, blockId_prd =allele_prd_blockId_prd.split(":")
        

        if allele_prd != "." and len(ref_allele)==1 and len(alt_allele)==1 : #  and blockId_prd != "."
            alleles_prd_ID = allele_prd+"|"+str(1-int(allele_prd))+":"+blockId_prd
            num_het+=1
        else:
            alleles_prd_ID = "./.:."
        
#         if allele_prd != "." and (len(ref_allele)>1 or len(alt_allele)>1):
#             num1+=1
           
        var_line_list_prd=var_line_list[:7] +['.', 'GT:PS', alleles_prd_ID]
        var_lines_list_prd.append(var_line_list_prd)
        

print("done",num_het,num1)


done 36947 0


In [13]:

vcf_out_address=vcf_file_address[:-4]+"_parent_v2d.vcf" # vcf_file_address+"_majority_v4b.vcf"

vcf_out_file=open(vcf_out_address,'w');

for header_line in header_lines_list[:-1]: # except last line should be edit for one sample not five
    vcf_out_file.write(header_line+"\n")

vcf_out_file.write(header_lines_list[-1][:51]+"\n")
    
for var_line_list in var_lines_list_prd:
    
    vcf_out_file.write("\t".join(var_line_list)+"\n")

vcf_out_file.close()


In [14]:
#16417020	1:16324005	0:16324005	1:16416076	1:16416076	1
block_id_s1='16324005'
block_id_s2='16324005'
block_id_s3='16416076'
block_id_s4='16416076'

print(block_id_s1 in reordered_block_i_list_s1)
print(block_id_s2 in reordered_block_i_list_s2)
print(block_id_s3 in reordered_block_i_list_s3)
print(block_id_s4 in reordered_block_i_list_s4)

True
False
True
True


In [14]:
#blockid_blocks_s1.index(block_id_s1), blockid_blocks_s2.index(block_id_s2), blockid_blocks_s3.index(block_id_s3)

# 